In [1]:
import Ipynb_importer
from a_csp_darknet53  import *

importing Jupyter notebook from a_csp_darknet53.ipynb


In [4]:
from functools import wraps

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Add, BatchNormalization, Concatenate,
                                     Conv2D, LeakyReLU, MaxPooling2D,Reshape,
                                     UpSampling2D, ZeroPadding2D)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

### 定义CBL 基本模块

In [17]:
def darknet_CBL(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    
    no_bias_kwargs = {'use_bias': False}  # 没懂为啥用 no_bias
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1)
    )

### 定义 yolo neck
<img src="https://pic3.zhimg.com/80/v2-5251e9c0784871a37c693d53f7d57f92_1440w.jpg" alt="img" style="zoom:50%;" />

In [ ]:
def yolov4_neck(inputs, num_anchors, num_classes):
    # 通过CSPdarknet_body 获得三个有效特征层
    feat1, feat2, feat3 = darknet_body(inputs)
    
    # p5  CBL*3
    P5 = darknet_CBL(512, (1, 1))(feat3)
    P5 = darknet_CBL(1024, (3, 3))(P5)
    P5 = darknet_CBL(512, (1, 1))(P5)
    # p5 使用了SPP结构，即不同尺度的最大池化后堆叠。
    maxpool1 = MaxPooling2D(pool_size=(13, 13), strides=(1, 1), padding='same')(P5)
    maxpool2 = MaxPooling2D(pool_size=(9, 9), strides=(1, 1), padding='same')(P5)
    maxpool3 = MaxPooling2D(pool_size=(5, 5), strides=(1, 1), padding='same')(P5)
    P5 = Concatenate()([maxpool1, maxpool2, maxpool3, P5])
    # p5  CBL*3
    P5 = darknet_CBL(512, (1, 1))(P5)
    P5 = darknet_CBL(1024, (3, 3))(P5)
    P5 = darknet_CBL(512, (1, 1))(P5)
    
    # p5 上采样与p4 实现 pan 结构，获得concatenate后的p4
    P5_upsample = compose(darknet_CBL(256, (1,1)), UpSampling2D(2))(P5)
    P4 = darknet_CBL(256, (1,1))(feat2)
    P4 = Concatenate()([P4, P5_upsample])
    
    # p4 CBL*5
    P4 = make_five_convs(P4,256)
    
    # p4 上采样与p3 实现 pan 结构，获得concatenate后的p3
    P4_upsample = compose(darknet_CBL(128, (1,1)), UpSampling2D(2))(P4)
    P3 = darknet_CBL(128, (1,1))(feat1)
    P3 = Concatenate()([P3, P4_upsample])
    # p3 CBL*5
    P3 = make_five_convs(P3,128)
    
    # ---------------------------------------------------------------------------------------------------------- #
    
    # p3 输出
    P3_output = darknet_CBL(256, (3,3))(P3)
    P3_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))(P3_output)

    # p3 下采样与 p4 实现 FPN，获得 concatenate 后的 p4
    P3_downsample = ZeroPadding2D(((1,0),(1,0)))(P3)
    P3_downsample = darknet_CBL(256, (3,3), strides=(2,2))(P3_downsample)
    P4 = Concatenate()([P3_downsample, P4])
    # p4 CBL*5
    P4 = make_five_convs(P4,256)
    # p4 输出
    P4_output = darknet_CBL(512, (3,3))(P4)
    P4_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))(P4_output)
    
    
    # p4 下采样与 p5 实现 FPN，获得 concatenate 后的 p5
    P4_downsample = ZeroPadding2D(((1,0),(1,0)))(P4)
    P4_downsample = darknet_CBL(512, (3,3), strides=(2,2))(P4_downsample)
    P5 = Concatenate()([P4_downsample, P5])
    # p5 CBL*5
    P5 = make_five_convs(P5,512)
    # p5输出
    P5_output = darknet_CBL(1024, (3,3))(P5)
    P5_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))(P5_output)

    return Model(inputs, [P5_output, P4_output, P3_output])

In [2]:
def yolov4_neck(inputs_shapes):
    """
    Implements the neck of YOLOv4, including the SPP and the modified PAN.

    Args:
        input_shapes (List[Tuple[int]]): List of 3 tuples, which are the output shapes of the backbone.
            For CSPDarknet53, those are: [(52, 52, 256), (26, 26, 512), (13, 13, 1024)] for a (416, 416) input.

    Returns:
        tf.keras.Model: Neck model
    """
    input_1 = tf.keras.Input(shape=filter(None, input_shapes[0]))  # 52* 52* 256
    input_2 = tf.keras.Input(shape=filter(None, input_shapes[1]))  # 26* 26* 512
    input_3 = tf.keras.Input(shape=filter(None, input_shapes[2]))  # 13* 13* 1024
    
    # p5  CBL*3
    P5 = darknet_CBL(512, (1, 1))(input_3)
    P5 = darknet_CBL(1024, (3, 3))(P5)
    P5 = darknet_CBL(512, (1, 1))(P5)
    # p5 使用了SPP结构，即不同尺度的最大池化后堆叠。
    maxpool1 = MaxPooling2D(pool_size=(13, 13), strides=(1, 1), padding='same')(P5)
    maxpool2 = MaxPooling2D(pool_size=(9, 9), strides=(1, 1), padding='same')(P5)
    maxpool3 = MaxPooling2D(pool_size=(5, 5), strides=(1, 1), padding='same')(P5)
    ssp = Concatenate()([maxpool1, maxpool2, maxpool3, P5])
    # p5  CBL*3
    P5 = darknet_CBL(512, (1, 1))(ssp)
    P5 = darknet_CBL(1024, (3, 3))(P5)
    output_3 = darknet_CBL(512, (1, 1))(P5)  # 13* 13* 512
    
    # output_3 上采样与p4 实现 pan 结构，获得concatenate后的p4
    P5_upsample = compose(darknet_CBL(256, (1,1)), UpSampling2D(2))(output_3)
    P4 = darknet_CBL(256, (1,1))(input_2)
    P4 = Concatenate()([P4, P5_upsample])
    
    # p4 CBL*5
    output_2 = make_five_convs(P4,256)  # 26* 26* 256
    
    # output_2 上采样与p3 实现 pan 结构，获得concatenate后的p3
    P4_upsample = compose(darknet_CBL(128, (1,1)), UpSampling2D(2))(output_2)
    P3 = darknet_CBL(128, (1,1))(input_1)
    P3 = Concatenate()([P3, P4_upsample])
    # p3 CBL*5
    output_1 = make_five_convs(P3,128)  # 52* 52* 128
    
    return tf.keras.Model(
        [input_1, input_2, input_3], [output_1, output_2, output_3], name="YOLOv4_neck"
    )